In [1]:
from sentence_transformers import SentenceTransformer, export_optimized_onnx_model, export_dynamic_quantized_onnx_model
model_id = "colorfulscoop/sbert-base-ja"
output_dir = model_id.replace("/", "-")
onnx_model = SentenceTransformer(model_id, backend="onnx", model_kwargs={"export": True})


/Users/s23326/Dev/DialogueMock/.venv/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
2024-10-17 15:07:39.577010 [W:onnxruntime:, helper.cc:82 IsInputSupported] CoreML does not support input dim > 16384. Input:embeddings.word_embeddings.weight, shape: {32000,768}
Saving the exported ONNX model is heavily recommended to avoid having to export it again. Do so with `model.push_to_hub('colorfulscoop/sbert-base-ja', create_pr=True)`.
2024-10-17 15:07:39.577692 [W:onnxruntime:, coreml_execution_provider.cc:107 GetCapability] CoreMLExecutionProvider::GetCapability, number of partitions supported by CoreML: 109 number of nodes in the graph: 802 number of nodes supported by CoreML: 542
2024-10-17 15:07:40.662973 [W:onnxruntime:, session_state.cc:1166 VerifyEachNodeIsAssignedToAnE

In [3]:
onnx_model.save_pretrained(output_dir)

In [5]:
for optimization_config in ["O1", "O2", "O3", "O4"]:
    export_optimized_onnx_model(
        onnx_model,
        optimization_config,
        output_dir,
    )

/Users/s23326/Dev/DialogueMock/.venv/lib/python3.10/site-packages/optimum/onnxruntime/configuration.py:779: FutureWarning: disable_embed_layer_norm will be deprecated soon, use disable_embed_layer_norm_fusion instead, disable_embed_layer_norm_fusion is set to True.
  warnings.warn(
There is no gpu for onnxruntime to do optimization.


In [6]:
for quantization_config in ['arm64', 'avx2', 'avx512', 'avx512_vnni']:
    export_dynamic_quantized_onnx_model(
        onnx_model,
        quantization_config,
        output_dir,
    )

In [13]:
import time
file_name = "onnx/model_qint8_avx512_vnni.onnx"
model = SentenceTransformer(output_dir, backend="onnx", model_kwargs={"file_name": file_name})
sentences = ["こんにちは", "おはようございます", "おやすみなさい", "さようなら"]
tic = time.time()
embeddings = model.encode(sentences)
toc = time.time() - tic
print(toc)
embeddings.shape

The ONNX file model_qint8_avx512_vnni.onnx is not a regular name used in optimum.onnxruntime, the ORTModel might not behave as expected.
2024-10-17 15:28:59.203091 [W:onnxruntime:, helper.cc:82 IsInputSupported] CoreML does not support input dim > 16384. Input:embeddings.word_embeddings.weight_quantized, shape: {32000,768}
2024-10-17 15:28:59.204027 [W:onnxruntime:, coreml_execution_provider.cc:107 GetCapability] CoreMLExecutionProvider::GetCapability, number of partitions supported by CoreML: 109 number of nodes in the graph: 1189 number of nodes supported by CoreML: 732
2024-10-17 15:29:00.443289 [W:onnxruntime:, session_state.cc:1166 VerifyEachNodeIsAssignedToAnEp] Some nodes were not assigned to the preferred execution providers which may or may not have an negative impact on performance. e.g. ORT explicitly assigns shape related ops to CPU to improve perf.
2024-10-17 15:29:00.443300 [W:onnxruntime:, session_state.cc:1168 VerifyEachNodeIsAssignedToAnEp] Rerunning with verbose outpu

0.26436686515808105


Context leak detected, msgtracer returned -1


(4, 768)

In [12]:
import time
file_names = [
    "model_O1.onnx",
    "model_O2.onnx",
    "model_O3.onnx",
    "model_O4.onnx",
    "onnx/model_qint8_avx512.onnx",
    "onnx/model_qint8_avx512_vnni.onnx",
    "onnx/model_qint8_avx2.onnx",
    "onnx/model_qint8_arm64.onnx",
]

n_iter = 30

for file_name in file_names:
    model = SentenceTransformer(output_dir, backend="onnx", model_kwargs={"file_name": file_name})
    time_list = []
    for i in range(n_iter):
        sentences = ["ランチって何時からでしたっけ"]
        tic = time.perf_counter()
        embeddings = model.encode(sentences)
        toc = time.perf_counter() - tic
        time_list.append(toc)
    print(f"Model: {file_name}, Average Time: {sum(time_list) / n_iter:.4f} s")

The ONNX file model_O1.onnx is not a regular name used in optimum.onnxruntime, the ORTModel might not behave as expected.
2024-10-17 15:25:31.093703 [W:onnxruntime:, helper.cc:82 IsInputSupported] CoreML does not support input dim > 16384. Input:embeddings.word_embeddings.weight, shape: {32000,768}
2024-10-17 15:25:31.094400 [W:onnxruntime:, coreml_execution_provider.cc:107 GetCapability] CoreMLExecutionProvider::GetCapability, number of partitions supported by CoreML: 109 number of nodes in the graph: 802 number of nodes supported by CoreML: 542
2024-10-17 15:25:32.187634 [W:onnxruntime:, session_state.cc:1166 VerifyEachNodeIsAssignedToAnEp] Some nodes were not assigned to the preferred execution providers which may or may not have an negative impact on performance. e.g. ORT explicitly assigns shape related ops to CPU to improve perf.
2024-10-17 15:25:32.187647 [W:onnxruntime:, session_state.cc:1168 VerifyEachNodeIsAssignedToAnEp] Rerunning with verbose output on a non-minimal build w

Model: model_O1.onnx, Average Time: 0.0507 s


2024-10-17 15:25:34.205846 [W:onnxruntime:, helper.cc:82 IsInputSupported] CoreML does not support input dim > 16384. Input:embeddings.word_embeddings.weight, shape: {32000,768}
2024-10-17 15:25:34.206262 [W:onnxruntime:, coreml_execution_provider.cc:107 GetCapability] CoreMLExecutionProvider::GetCapability, number of partitions supported by CoreML: 61 number of nodes in the graph: 493 number of nodes supported by CoreML: 209
2024-10-17 15:25:34.833307 [W:onnxruntime:, session_state.cc:1166 VerifyEachNodeIsAssignedToAnEp] Some nodes were not assigned to the preferred execution providers which may or may not have an negative impact on performance. e.g. ORT explicitly assigns shape related ops to CPU to improve perf.
2024-10-17 15:25:34.833316 [W:onnxruntime:, session_state.cc:1168 VerifyEachNodeIsAssignedToAnEp] Rerunning with verbose output on a non-minimal build will show node assignments.
Context leak detected, msgtracer returned -1
Context leak detected, msgtracer returned -1
Contex

Model: model_O2.onnx, Average Time: 0.0326 s


2024-10-17 15:25:36.287617 [W:onnxruntime:, helper.cc:82 IsInputSupported] CoreML does not support input dim > 16384. Input:embeddings.word_embeddings.weight, shape: {32000,768}
2024-10-17 15:25:36.288000 [W:onnxruntime:, coreml_execution_provider.cc:107 GetCapability] CoreMLExecutionProvider::GetCapability, number of partitions supported by CoreML: 61 number of nodes in the graph: 493 number of nodes supported by CoreML: 209
2024-10-17 15:25:36.926905 [W:onnxruntime:, session_state.cc:1166 VerifyEachNodeIsAssignedToAnEp] Some nodes were not assigned to the preferred execution providers which may or may not have an negative impact on performance. e.g. ORT explicitly assigns shape related ops to CPU to improve perf.
2024-10-17 15:25:36.926913 [W:onnxruntime:, session_state.cc:1168 VerifyEachNodeIsAssignedToAnEp] Rerunning with verbose output on a non-minimal build will show node assignments.
Context leak detected, msgtracer returned -1
Context leak detected, msgtracer returned -1
The ON

...

Model: model_O4.onnx, Average Time: 0.0229 s


2024-10-17 15:25:39.873168 [W:onnxruntime:, helper.cc:82 IsInputSupported] CoreML does not support input dim > 16384. Input:embeddings.word_embeddings.weight_quantized, shape: {32000,768}
2024-10-17 15:25:39.874000 [W:onnxruntime:, coreml_execution_provider.cc:107 GetCapability] CoreMLExecutionProvider::GetCapability, number of partitions supported by CoreML: 109 number of nodes in the graph: 1189 number of nodes supported by CoreML: 732
2024-10-17 15:25:40.984518 [W:onnxruntime:, session_state.cc:1166 VerifyEachNodeIsAssignedToAnEp] Some nodes were not assigned to the preferred execution providers which may or may not have an negative impact on performance. e.g. ORT explicitly assigns shape related ops to CPU to improve perf.
2024-10-17 15:25:40.984528 [W:onnxruntime:, session_state.cc:1168 VerifyEachNodeIsAssignedToAnEp] Rerunning with verbose output on a non-minimal build will show node assignments.
Context leak detected, msgtracer returned -1
Context leak detected, msgtracer return

Model: onnx/model_qint8_avx512.onnx, Average Time: 0.0297 s


2024-10-17 15:25:42.377259 [W:onnxruntime:, helper.cc:82 IsInputSupported] CoreML does not support input dim > 16384. Input:embeddings.word_embeddings.weight_quantized, shape: {32000,768}
2024-10-17 15:25:42.378080 [W:onnxruntime:, coreml_execution_provider.cc:107 GetCapability] CoreMLExecutionProvider::GetCapability, number of partitions supported by CoreML: 109 number of nodes in the graph: 1189 number of nodes supported by CoreML: 732
2024-10-17 15:25:43.500035 [W:onnxruntime:, session_state.cc:1166 VerifyEachNodeIsAssignedToAnEp] Some nodes were not assigned to the preferred execution providers which may or may not have an negative impact on performance. e.g. ORT explicitly assigns shape related ops to CPU to improve perf.
2024-10-17 15:25:43.500044 [W:onnxruntime:, session_state.cc:1168 VerifyEachNodeIsAssignedToAnEp] Rerunning with verbose output on a non-minimal build will show node assignments.
Context leak detected, msgtracer returned -1
Context leak detected, msgtracer return

Fail: [ONNXRuntimeError] : 1 : FAIL : Non-zero status code returned while running MatMulInteger node. Name:'/encoder/layer.1/attention/self/MatMul_1_quant' Status Message: matmul_helper.h:162 Compute MatMul dimension mismatch

In [ ]:
import os
import numpy as np
from sentence_transformers import SentenceTransformer
import time
from src.utils import get_custom_logger
logger = get_custom_logger("faq")

model_dir = "/Users/s23326/Dev/DialogueMock/colorfulscoop-sbert-base-ja/"
model_file = "onnx/model_qint8_avx512_vnni.onnx"

class SbertFAQ:
    def __init__(self):
        
        # モデルをロード
        logger.info("Loading ONNX model...")
        start_time = time.time()
        self.model = SentenceTransformer(model_dir, backend="onnx", model_kwargs={"model_file": model_file})
        logger.info(f"Model loaded in {time.time() - start_time:.2f} seconds.")
        
        # FAQデータを設定
        self.faq_dict = {
            "営業時間": [
                "ランチの営業時間は11:00から15:00です",
                "ディナーの営業時間は17:00から23:00です",
                "営業時間は日によって異なる場合がありますので、お問い合わせください"
            ],
            "駐車場": [
                "駐車場は2台まで停められます",
                "近隣にコインパーキングもございます"
            ],
            "席代": [
                "ランチは席代がかかりませんが、ディナーは席代がかかります",
                "ディナータイムの席代はお一人様500円です"
            ],
            "予約": [
                "ランチは予約できません",
                "ディナーのご予約はお電話で承っております"
            ]
        }
        
        # 質問キーワードと回答を準備
        self.questions = list(self.faq_dict.keys())
        self.question_embeddings = self.encode_sentences(self.questions)
        
        # 各質問キーワードに対応する回答をベクトル化
        self.answers_list = []
        self.answer_embeddings = []
        for key in self.questions:
            answers = self.faq_dict[key]
            embeddings = self.encode_sentences(answers)
            self.answers_list.append(answers)
            self.answer_embeddings.append(embeddings)
    
    def encode_sentences(self, sentences):
        # 文をエンコード（ベクトル化）
        embeddings = self.model.encode(sentences)
        return embeddings
    
    def get_answer(self, query):
        # ユーザーの質問をエンコード
        query_embedding = self.encode_sentences([query])[0]
        
        # 質問キーワードとの類似度を計算
        similarities = np.dot(self.question_embeddings, query_embedding) / (np.linalg.norm(self.question_embeddings, axis=1) * np.linalg.norm(query_embedding))
        
        max_sim = similarities.max()
        threshold = 0.5  # 閾値を適切に設定してください
        if max_sim > threshold:
            best_question_idx = similarities.argmax()
            best_question = self.questions[best_question_idx]
            
            # 対応する回答リスト内で最適な回答を選択
            answer_embeddings = self.answer_embeddings[best_question_idx]
            answer_similarities = np.dot(answer_embeddings, query_embedding) / (np.linalg.norm(answer_embeddings, axis=1) * np.linalg.norm(query_embedding))
            best_answer_idx = answer_similarities.argmax()
            best_answer = self.answers_list[best_question_idx][best_answer_idx]
            return best_answer
        else:
            return "申し訳ありませんが、そのご質問にはお答えできません。"

if __name__ == "__main__":
    # クラスのインスタンスを作成
    faq = SbertFAQ()
    
    # テスト用の質問リスト
    queries = [
        "ランチって設置代って追加でかかったりしますか",  # 音声認識誤りを含む質問
        "ディナー予約できますか？",
        "駐車場はありますか？",
        "ランチの時間を教えて下さい",
    ]
    
    # 質問に対する回答を取得
    for query in queries:
        answer = faq.get_answer(query)
        print(f"質問: {query}")
        print(f"回答:\n{answer}\n")